# Fraud Detection - Silver Layer Transformation

## Purpose
Transform bronze layer data into clean, business-ready silver tables with:
- Data type corrections (string amounts → decimals)
- Normalization (uppercase flags, trimmed strings)
- Deduplication
- **Train/test split based on fraud labels (happens here in Silver)**

## Silver Tables Created
1. `transactions_silver` - All cleaned transactions
2. `labels_silver` - Fraud labels (deduplicated)
3. `tx_train_silver` - Labeled transactions (for training) **← Split happens here**
4. `tx_score_silver` - Unlabeled transactions (for scoring) **← Split happens here**
5. `cards_silver` - Card dimension (sensitive fields removed)
6. `users_silver` - User dimension (monetary fields as decimals)
7. `mcc_dim_silver` - Merchant category codes

## Data Flow
Bronze Tables → Clean & Transform → Silver Tables → **Train/Score Split**

---
## 1. Transform Transactions
Clean and standardize transaction data from bronze layer.

In [0]:
# Load from single bronze table (no union needed)
transactions = spark.table("workspace.fraud.transactions_bronze")

In [0]:
transactions.printSchema()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

transactions_clean = (
    transactions
    .withColumn("amount",
        F.regexp_replace(F.col("amount"), "[$]", "")
    )
    .withColumn("amount",
        F.col("amount").cast(DecimalType(12,2))
    )
)

In [0]:
transactions_clean = transactions_clean.withColumn(
    "mcc",
    F.col("mcc").cast("int")
)

In [0]:
transactions_clean.printSchema()

In [0]:
# check nulls
transactions_clean.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in transactions_clean.columns]
).show()

In [0]:
# indicate "None" if no error
transactions_clean = transactions_clean.withColumn(
    "errors",
    F.when(F.col("errors").isNull(), F.lit("None"))
     .otherwise(F.col("errors"))
)

In [0]:
# check amount distribution sanity
transactions_clean.select(
    F.min("amount"),
    F.max("amount"),
    F.avg("amount")
).show()

In [0]:
# check skewness
transactions_clean.select(
    F.expr("percentile_approx(amount, array(0.5, 0.9, 0.99))")
).show(truncate=False)

In [0]:
from pyspark.sql import functions as F

transactions_silver = (
    transactions_clean
    .withColumn("use_chip", F.upper(F.trim(F.col("use_chip"))))  # normalize
)

transactions_silver.write.mode("overwrite").option("overwriteSchema", "true").format("delta") \
    .saveAsTable("workspace.fraud.transactions_silver")

print(f"Created transactions_silver with {transactions_silver.count():,} rows")
spark.table("workspace.fraud.transactions_silver").printSchema()

---
## 2. Transform Fraud Labels
Load and clean fraud labels from bronze layer.

In [0]:
# Load labels from bronze layer
labels_map = spark.table("workspace.fraud.labels_map")

labels_map.show(5, truncate=False)
print("labels count:", labels_map.count())
print("label distribution:")
labels_map.groupBy("label").count().show()

In [0]:
labels_silver = (
    spark.table("workspace.fraud.labels_map")
    .select(F.col("tx_id").cast("string").alias("tx_id"), F.col("label"))
    .dropDuplicates(["tx_id"])
)

labels_silver.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.fraud.labels_silver")


In [0]:
from pyspark.sql import functions as F

spark.table("workspace.fraud.labels_silver").groupBy("label").count().show()
spark.table("workspace.fraud.labels_silver").select(F.countDistinct("tx_id")).show()

---
## 3. Create Train/Score Split
Split transactions into labeled (training) and unlabeled (scoring) datasets.

In [0]:
tx_train_silver = (
  spark.table("workspace.fraud.transactions_silver")
    .join(spark.table("workspace.fraud.labels_silver"), on="tx_id", how="inner")
)

tx_train_silver.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("workspace.fraud.tx_train_silver")

In [0]:
spark.sql("DESCRIBE TABLE workspace.fraud.tx_train_silver").show(truncate=False)

# training set size
tx_train_silver.select("tx_id").distinct().count()


In [0]:
tx_score_silver = (
  spark.table("workspace.fraud.transactions_silver")
    .join(spark.table("workspace.fraud.labels_silver"), on="tx_id", how="left_anti")
)

tx_score_silver.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("workspace.fraud.tx_score_silver")

In [0]:
spark.sql("DESCRIBE TABLE workspace.fraud.tx_score_silver").show(truncate=False)

# test set size
tx_score_silver.select("tx_id").distinct().count()

In [0]:
spark.table("workspace.fraud.tx_score_silver").show(5, truncate=False)
spark.table("workspace.fraud.tx_score_silver").describe().show(truncate=False)

---
## 4. Transform Cards Dimension
Clean card data.

In [0]:
from pyspark.sql import functions as F

cards_bronze = spark.table("workspace.fraud.cards_bronze")

cards_silver = (
    cards_bronze
    .withColumn("has_chip", F.upper(F.trim(F.col("has_chip"))))  # normalize YES/NO
    .withColumn("card_on_dark_web", F.upper(F.trim(F.col("card_on_dark_web"))))  # normalize
    .dropDuplicates(["id"])
)

cards_silver.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.fraud.cards_silver")

spark.table("workspace.fraud.cards_silver").printSchema()
spark.table("workspace.fraud.cards_silver").show(5, truncate=False)

---
## 5. Transform Users Dimension
Clean user data and convert monetary fields to decimals.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

users_bronze = spark.table("workspace.fraud.users_bronze")

def money_to_decimal(colname):
    # remove $ and commas, cast to decimal
    return F.regexp_replace(F.col(colname), r"[\$,]", "").cast(DecimalType(18, 2))

users_silver = (
    users_bronze
    .withColumn("per_capita_income", money_to_decimal("per_capita_income"))
    .withColumn("yearly_income", money_to_decimal("yearly_income"))
    .withColumn("total_debt", money_to_decimal("total_debt"))
    .dropDuplicates(["id"])
)

users_silver.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.fraud.users_silver")

spark.table("workspace.fraud.users_silver").printSchema()
spark.table("workspace.fraud.users_silver").show(5, truncate=False)


In [0]:
%sql
-- Should be unique in users_silver
SELECT COUNT(*) cnt, COUNT(DISTINCT id) distinct_cnt
FROM workspace.fraud.users_silver;

In [0]:
%sql
-- Should be unique in cards_silver (id is card_id)
SELECT COUNT(*) cnt, COUNT(DISTINCT id) distinct_cnt
FROM workspace.fraud.cards_silver;

In [0]:
%sql
-- transactions_silver: tx_id should be unique (or close to it)
SELECT COUNT(*) cnt, COUNT(DISTINCT tx_id) distinct_cnt
FROM workspace.fraud.transactions_silver;

---
## 6. Transform MCC Dimension
Clean merchant category code lookup table.

In [0]:
from pyspark.sql import functions as F

mcc_dim_silver = (
    spark.table("workspace.fraud.mcc_dim_bronze")
    .select(
        F.col("mcc").cast("int").alias("mcc"),
        F.trim(F.col("mcc_description")).alias("mcc_description")
    )
    .filter(F.col("mcc").isNotNull())
    .dropDuplicates(["mcc"])
)

mcc_dim_silver.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.fraud.mcc_dim_silver")

# verify
spark.table("workspace.fraud.mcc_dim_silver").printSchema()
spark.table("workspace.fraud.mcc_dim_silver").show(5, truncate=False)
spark.sql("""
  SELECT COUNT(*) AS rows, COUNT(DISTINCT mcc) AS distinct_mcc
  FROM workspace.fraud.mcc_dim_silver
""").show()

---
## 7. Verify Silver Tables
Confirm all silver tables were created successfully with expected row counts.

In [0]:
# Summary of all silver tables
silver_tables = [
    "transactions_silver",
    "labels_silver",
    "tx_train_silver",
    "tx_score_silver",
    "cards_silver",
    "users_silver",
    "mcc_dim_silver"
]

print("Silver Layer Tables Summary")
print("=" * 60)

for table in silver_tables:
    try:
        count = spark.sql(f"SELECT COUNT(*) as cnt FROM workspace.fraud.{table}").collect()[0]['cnt']
        print(f"{table:25} {count:>15,} rows")
    except Exception as e:
        print(f"{table:25} ERROR: {str(e)[:30]}")

print("=" * 60)